# Pseudo dataset
* Get sample data [here](https://github.com/fuyu-quant/IBLM/tree/main/datasets).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/IBLM/blob/main/examples/iblmodel_pseudodata.ipynb)

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/IBLM.git

In [2]:
import pkg_resources
print(pkg_resources.get_distribution('IBLM').version)

0.0.21


### Training

In [3]:
import pandas as pd
import numpy as np
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import IBLMClassifier


import os
#os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [25]:
file_name = 'pseudodata_4f_train.csv'
#file_name = 'pseudodata_8f_train.csv'
#file_name = 'pseudodata_12f_train.csv'

#df = pd.read_csv(f'/content/{file_name}}')
df = pd.read_csv(f'../datasets/{file_name}')
x_train = df.drop(df.columns[-1], axis=1)
y_train = df[df.columns[-1]]

In [36]:
llm_model_name = 'gpt-4'

params = {
    'columns_name': True
    }

iblm = IBLMClassifier(llm_model_name=llm_model_name, params=params)

In [ ]:
#file_path = '/content/'
file_path = './model_code/'

model = iblm.fit(x_train, y_train, model_name = 'pseudodata', file_path=file_path)

In [47]:
# Code of the model created
print(model)

import numpy as np

def predict(x):
    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Do not change the code before this point.
        # Please describe the process required to make the prediction below.

        # Calculate the weighted sum of the features
        weighted_sum = row['A'] * 0.25 + row['B'] * 0.35 + row['C'] * 0.2 + row['D'] * 0.15

        # Apply the sigmoid function to the weighted sum to get the probability
        y = 1 / (1 + np.exp(-weighted_sum))

        # Do not change the code after this point.
        output.append(y)
    return np.array(output)


### Prediction

In [43]:
file_name = 'pseudodata_4f_test.csv'
#file_name = 'pseudodata_8f_test.csv'
#file_name = 'pseudodata_12f_test.csv'

#df = pd.read_csv(f'/content/{file_name}}')
df = pd.read_csv(f'../datasets/{file_name}')
x_test = df.drop(df.columns[-1], axis=1)
y_test = df[df.columns[-1]]

In [44]:
y_proba = iblm.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [45]:
accuracy = round(accuracy_score(y_test, y_pred),4)
print(f'Accuracy: {accuracy}')

# Precision
precision = round(precision_score(y_test, y_pred),4)
print(f'Precision: {precision}')

# Recall
recall = round(recall_score(y_test, y_pred),4)
print(f'Recall: {recall}')

# F1 score
f1 = round(f1_score(y_test, y_pred),4)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = round(roc_auc_score(y_test, y_proba),4)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.6077
Precision: 0.6504
Recall: 0.4863
F1 score: 0.5565
ROC-AUC: 0.7625


### Prediction from external files


In [ ]:
import pseudodata

y_proba = pseudodata.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [20]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.415
Precision: 0.43103448275862066
Recall: 0.49504950495049505
F1 score: 0.4608294930875576
ROC-AUC: 0.35973597359735976


### Interpretation of results

In [57]:
description = iblm.interpret()

Tokens Used: 881
	Prompt Tokens: 537
	Completion Tokens: 344
Successful Requests: 1
Total Cost (USD): $0.036750000000000005


In [58]:
print(description)

- Data preprocessing:
    - Fill missing 'age' values with the median age.
    - Fill missing 'fare' values with the median fare.
    - Fill missing 'embarked' values with the mode (most frequent) of the 'embarked' column.

- Feature creation:
    - Create a new binary feature 'is_female' based on the 'sex' column.
    - Create a new binary feature 'is_child' based on the 'age' column.
    - Create a new binary feature 'is_adult_male' based on the 'adult_male' column.
    - Create a new binary feature 'is_alone' based on the 'alone' column.
    - Create new binary features 'is_first_class', 'is_second_class', and 'is_third_class' based on the 'pclass' column.
    - Create new binary features 'embarked_C', 'embarked_Q', and 'embarked_S' based on the 'embarked' column.

- Prediction logic:
    - Initialize a variable 'y' to 0.
    - Add or subtract weights to 'y' based on the created binary features.
    - Apply the logistic function (sigmoid) to 'y' to get the final prediction.
    - Ap

### Creating Multiple Models

In [46]:
#file_path = '/content/'
file_path = './code_model/'

for i in range(30):
    model = iblm.fit(x_train, y_train, model_name = f'pseudodata_{i}_', file_path=file_path)

> Start of model creating.
Tokens Used: 7629
	Prompt Tokens: 7470
	Completion Tokens: 159
Successful Requests: 1
Total Cost (USD): $0.23364
> Start of model creating.
Tokens Used: 7629
	Prompt Tokens: 7470
	Completion Tokens: 159
Successful Requests: 1
Total Cost (USD): $0.23364
> Start of model creating.
Tokens Used: 7618
	Prompt Tokens: 7470
	Completion Tokens: 148
Successful Requests: 1
Total Cost (USD): $0.23298
> Start of model creating.
Tokens Used: 7629
	Prompt Tokens: 7470
	Completion Tokens: 159
Successful Requests: 1
Total Cost (USD): $0.23364
> Start of model creating.
Tokens Used: 7642
	Prompt Tokens: 7470
	Completion Tokens: 172
Successful Requests: 1
Total Cost (USD): $0.23442
> Start of model creating.
Tokens Used: 7599
	Prompt Tokens: 7470
	Completion Tokens: 129
Successful Requests: 1
Total Cost (USD): $0.23184
> Start of model creating.
Tokens Used: 7629
	Prompt Tokens: 7470
	Completion Tokens: 159
Successful Requests: 1
Total Cost (USD): $0.23364
> Start of model cre